# Loading PDF file

In [1]:
# Loading PyPDFLoader library
from langchain_community.document_loaders import PyPDFLoader

# Loading the PyPDFLoader with pdf file
loader_pdf = PyPDFLoader('Introduction_to_Data_and_Data_Science.pdf')

# Loading the pages
loader_pages = loader_pdf.load()

for i in loader_pages:
    i.page_content = ' '.join(i.page_content.split())

# Displaying the sample
loader_pages[0].page_content

'Analysis vs Analytics Alright! So… Let’s discuss the not-so-obvious differences between the terms analysis and analytics. Due to the similarity of the words, some people believe they share the same meaning, and thus use them interchangeably. Technically, this isn’t correct. There is, in fact, a distinct difference between the two. And the reason for one often being used instead of the other is the lack of a transparent understanding of both. So, let’s clear this up, shall we? First, we will start with analysis. Consider the following… You have a huge dataset containing data of various types. Instead of tackling the entire dataset and running the risk of becoming overwhelmed, you separate it into easier to digest chunks and study them individually and examine how they relate to other parts. And that’s analysis in a nutshell. One important thing to remember, however, is that you perform analyses on things that have already happened in the past. Such as using an analysis to explain how a

# Loading Docx file

In [1]:
# Loading the docx2txt library
from langchain_community.document_loaders import Docx2txtLoader

# Loading the Docx2txt with docx file
loader_docx = Docx2txtLoader('Introduction_to_Data_and_Data_Science_2.docx')

# Loading the pages
pages = loader_docx.load()

# # Removing new line character
# for i in range(len(pages)):
#     pages[i].page_content = ' '.join(pages[i].page_content.split())

# displaying the page content
pages[0].page_content

"# Introduction to Data Science\n\n## Analysis vs Analytics\n\nAlright! So…\nLet’s discuss the not-so-obvious differences\nbetween the terms analysis and analytics.\nDue to the similarity of the words, some people\nbelieve they share the same meaning, and thus\nuse them interchangeably. Technically, this\nisn’t correct. There is, in fact, a distinct\ndifference between the two. And the reason\nfor one often being used instead of the other\nis the lack of a transparent understanding\nof both.\nSo, let’s clear this up, shall we?\nFirst, we will start with analysis.\nConsider the following…\nYou have a huge dataset containing data of\nvarious types. Instead of tackling the entire\ndataset and running the risk of becoming overwhelmed,\nyou separate it into easier to digest chunks\nand study them individually and examine how\nthey relate to other parts. And that’s analysis\nin a nutshell.\nOne important thing to remember, however,\nis that you perform analyses on things that\nhave already h

# Text Splitter

In [28]:
# Loading the document
from langchain_text_splitters.character import CharacterTextSplitter

# Printing the page characters
print(len(pages[0].page_content))

# Defining the text splitter
text_splitter = CharacterTextSplitter(separator='.', 
                                      chunk_size=500,
                                      chunk_overlap=50)

# Page chunk split
pages_char_split = text_splitter.split_documents(pages)

# Printing first chunk
len(pages_char_split)

8300


21

### Markdown Text Splitter

In [4]:
# Loading the document
from langchain_text_splitters.markdown import MarkdownHeaderTextSplitter

# Printing the page characters
print(len(pages[0].page_content))

# Defining the text splitter
md_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=[('#', 'Course Title'), ('##', 'Section Title')])

# Page chunk split
pages_md_split = md_splitter.split_text(pages[0].page_content)

# Printing first chunk
pages_md_split

8300


[Document(metadata={'Course Title': 'Introduction to Data Science', 'Section Title': 'Analysis vs Analytics'}, page_content="Alright! So…\nLet’s discuss the not-so-obvious differences\nbetween the terms analysis and analytics.\nDue to the similarity of the words, some people\nbelieve they share the same meaning, and thus\nuse them interchangeably. Technically, this\nisn’t correct. There is, in fact, a distinct\ndifference between the two. And the reason\nfor one often being used instead of the other\nis the lack of a transparent understanding\nof both.\nSo, let’s clear this up, shall we?\nFirst, we will start with analysis.\nConsider the following…\nYou have a huge dataset containing data of\nvarious types. Instead of tackling the entire\ndataset and running the risk of becoming overwhelmed,\nyou separate it into easier to digest chunks\nand study them individually and examine how\nthey relate to other parts. And that’s analysis\nin a nutshell.\nOne important thing to remember, however

# Text Embedding with OpenAI

In [1]:
%load_ext dotenv
%dotenv

In [21]:
# Loading the docx2txt library
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters.character import CharacterTextSplitter
from langchain_text_splitters.markdown import MarkdownHeaderTextSplitter

from langchain_openai import OpenAIEmbeddings

# Loading the Docx2txt with docx file
loader_docx = Docx2txtLoader('Introduction_to_Data_and_Data_Science_2.docx')

# Loading the pages
pages = loader_docx.load()

# Defining the Markdown text splitter
md_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=[('#', 'Course Title'), ('##', 'Section Title')])

# Text chunks based on markdown
pages_md_split = md_splitter.split_text(pages[0].page_content)

# Removing new line characters
for i in pages_md_split:
    i.page_content = ' '.join(i.page_content.split())

# Defining the character text splitter
text_splitter = CharacterTextSplitter(separator='.', 
                                      chunk_size=500,
                                      chunk_overlap=50)

# Character chunks
pages_char_split = text_splitter.split_documents(pages_md_split)

# Defining the embedding model
embedding = OpenAIEmbeddings(model='text-embedding-ada-002')

# Creating the vector embeddings
vector1 = embedding.embed_query(pages_char_split[3].page_content)
vector2 = embedding.embed_query(pages_char_split[5].page_content)
vector3 = embedding.embed_query(pages_char_split[18].page_content)

# Length on vectors
# len(vector1), len(vector2), len(vector3)

# Relations of vectors with each other 
round(np.dot(vector1, vector2), 3), round(np.dot(vector1, vector3), 3), round(np.dot(vector2, vector3), 3)

(np.float64(0.879), np.float64(0.8), np.float64(0.793))

In [22]:
len(pages_char_split)

20

# Vectorstore with Chroma

In [1]:
%load_ext dotenv
%dotenv

In [2]:
# Importing libraries
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters.character import CharacterTextSplitter
from langchain_text_splitters.markdown import MarkdownHeaderTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

# Loading the Docx2txt with docx file
loader_docx = Docx2txtLoader('Introduction_to_Data_and_Data_Science_2.docx')

# Loading the pages
pages = loader_docx.load()

# Defining the Markdown text splitter
md_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=[('#', 'Course Title'), ('##', 'Section Title')])

# Text chunks based on markdown
pages_md_split = md_splitter.split_text(pages[0].page_content)

# Removing new line characters
for i in pages_md_split:
    i.page_content = ' '.join(i.page_content.split())

# Defining the character text splitter
text_splitter = CharacterTextSplitter(separator='.', 
                                      chunk_size=500,
                                      chunk_overlap=50)

# Character chunks
pages_char_split = text_splitter.split_documents(pages_md_split)

# Defining the embedding model
embedding = OpenAIEmbeddings(model='text-embedding-ada-002')

# Vectorspace Creations
vectorstore = Chroma.from_documents(documents=pages_char_split,
                                    embedding=embedding,
                                    persist_directory='./intro-to-ds-lectures')

vectorstore_from_direcory = Chroma(persist_directory='./intro-to-ds-lectures',
                                   embedding_function=embedding)

# Retrieval - Similarity Search

In [15]:
%load_ext dotenv
%dotenv

In [16]:
# Importing libraries
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain_core.documents import Document

# Defining embedding
embedding = OpenAIEmbeddings(model='text-embedding-ada-002')

# Vectorstore 
vectorstore = Chroma(persist_directory='./intro-to-ds-lectures',
                                   embedding_function=embedding)

# Adding Duplicate 
added_document = Document(page_content="Alright! So… Let’s discuss the not-so-obvious differences between the terms analysis and analytics. Due to the similarity of the words, some people believe they share the same meaning, and thus use them interchangeably. Technically, this isn’t correct. There is, in fact, a distinct difference between the two. And the reason for one often being used instead of the other is the lack of a transparent understanding of both. So, let’s clear this up, shall we? First, we will start with analysis'",
                          metadata = {'Course Title': 'Introduction to Data Science',
                                       'Section Title': 'Programming Languages & Software Employed in Data Science - All the Tools You Need'})
vectorstore.add_documents([added_document])

['c3f34ee4-2d97-4d13-828f-3bcae4e79314']

In [10]:
vectorstore.delete("e53be856-b41a-4c3a-b9f2-15152c6c8cb1")

In [13]:
vectorstore.get("1b942c45-c163-440b-94e0-57c56a1efacc")

{'ids': ['1b942c45-c163-440b-94e0-57c56a1efacc'],
 'embeddings': None,
 'documents': ["Alright! So… Let’s discuss the not-so-obvious differences between the terms analysis and analytics. Due to the similarity of the words, some people believe they share the same meaning, and thus use them interchangeably. Technically, this isn’t correct. There is, in fact, a distinct difference between the two. And the reason for one often being used instead of the other is the lack of a transparent understanding of both. So, let’s clear this up, shall we? First, we will start with analysis'"],
 'uris': None,
 'included': ['metadatas', 'documents'],
 'data': None,
 'metadatas': [{'Course Title': 'Introduction to Data Science',
   'Section Title': 'Programming Languages & Software Employed in Data Science - All the Tools You Need'}]}

In [17]:
question = "What are the programming languages do data scientists use?"

retrieved_docs = vectorstore.similarity_search(query=question, k=5)

for i in retrieved_docs:
    print(f"{i.page_content}\n------------\n{i.metadata['Section Title']}\n\n")

What about big data? Apart from R and Python, people working in this area are often proficient in other languages like Java or Scala. These two have not been developed specifically for doing statistical analyses, however they turn out to be very useful when combining data from multiple sources. All right! Let’s finish off with machine learning. When it comes to machine learning, we often deal with big data
------------
Programming Languages & Software Employed in Data Science - All the Tools You Need


Thus, we need a lot of computational power, and we can expect people to use the languages similar to those in the big data column. Apart from R, Python, and MATLAB, other, faster languages are used like Java, JavaScript, C, C++, and Scala. Cool. What we said may be wonderful, but that’s not all! By using one or more programming languages, people create application software or, as they are sometimes called, software solutions, that are adjusted for specific business needs
------------
Pro

In [23]:
question = "What software do data scientists use?"

retrieved_docs = vectorstore.max_marginal_relevance_search(
    query=question, 
    k=3,
    lambda_mult=0.9,
    filter={"Section Title": "Programming Languages & Software Employed in Data Science - All the Tools You Need" }
)

for i in retrieved_docs:
    print(f"{i.page_content}\n------------\n{i.metadata['Section Title']}\n\n")

As you can see from the infographic, R, and Python are the two most popular tools across all columns. Their biggest advantage is that they can manipulate data and are integrated within multiple data and data science software platforms. They are not just suitable for mathematical and statistical computations. In other words, R, and Python are adaptable. They can solve a wide variety of business and data-related problems from beginning to the end
------------
Programming Languages & Software Employed in Data Science - All the Tools You Need


Alright! So… How are the techniques used in data, business intelligence, or predictive analytics applied in real life? Certainly, with the help of computers. You can basically split the relevant tools into two categories—programming languages and software. Knowing a programming language enables you to devise programs that can execute specific operations. Moreover, you can reuse these programs whenever you need to execute the same action
------------